# SAE Session 5: Advanced Robot Control & Optimization
## Industry 4.0 Smart Manufacturing Cell - OpenManipulator-X

**Duration**: 8 hours  
**Prerequisites**: Sessions 1-4 completed  
**Objectives**:
- Implement velocity control and acceleration limits
- Optimize cycle time while respecting constraints
- Implement singularity avoidance
- Add collision detection and avoidance
- Develop energy-efficient trajectories

---

## Part 1: Velocity and Acceleration Control (2 hours)

### Theory

In industrial applications, respecting velocity and acceleration limits is crucial for:
- **Safety**: Preventing dangerous robot movements
- **Accuracy**: Reducing overshoot and vibrations
- **Equipment lifespan**: Avoiding excessive wear
- **Energy efficiency**: Smooth motions consume less power

**Constraints**:
- Joint velocity limits: $|\dot{q}_i| \leq \dot{q}_{i,max}$
- Joint acceleration limits: $|\ddot{q}_i| \leq \ddot{q}_{i,max}$
- Cartesian velocity: $|\dot{x}| \leq v_{max}$
- Jerk limits: $|\dddot{q}_i| \leq j_{max}$ (smoothness)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline, make_interp_spline
from roboticstoolbox import DHRobot, RevoluteDH
import spatialmath as sm

# OpenManipulator-X definition
class OpenManipulatorX(DHRobot):
    def __init__(self):
        # DH parameters: [theta, d, a, alpha]
        L1 = RevoluteDH(d=0.077, a=0, alpha=np.pi/2, qlim=np.deg2rad([-180, 180]))
        L2 = RevoluteDH(d=0, a=0.130, alpha=0, offset=-np.pi/2, qlim=np.deg2rad([-90, 90]))
        L3 = RevoluteDH(d=0, a=0.124, alpha=0, qlim=np.deg2rad([-90, 90]))
        L4 = RevoluteDH(d=0, a=0.126, alpha=0, qlim=np.deg2rad([-180, 180]))
        
        super().__init__([L1, L2, L3, L4], name='OpenManipulatorX')
        
        # Physical limits
        self.qd_max = np.deg2rad([100, 100, 100, 100])  # max velocity (rad/s)
        self.qdd_max = np.deg2rad([300, 300, 300, 300]) # max acceleration (rad/s²)
        self.jerk_max = np.deg2rad([1000, 1000, 1000, 1000]) # max jerk (rad/s³)
        self.v_cart_max = 0.2  # max Cartesian velocity (m/s)

robot = OpenManipulatorX()
print(robot)

### Exercise 5.1: Velocity Profile Analysis

Implement functions to check if a trajectory respects velocity and acceleration limits.

In [ ]:
def check_velocity_limits(q_traj, t_traj, qd_max):
    """
    Check if trajectory respects velocity limits.
    
    Parameters:
    -----------
    q_traj : ndarray (N, n_joints)
        Joint positions trajectory
    t_traj : ndarray (N,)
        Time vector
    qd_max : ndarray (n_joints,)
        Maximum joint velocities
    
    Returns:
    --------
    is_valid : bool
        True if all velocities respect limits
    qd_traj : ndarray (N, n_joints)
        Computed velocities
    violations : list
        List of (time_idx, joint_idx, velocity, limit) for violations
    """
    # TODO: Implement velocity checking
    # Hint: Use numerical differentiation: qd[i] ≈ (q[i+1] - q[i]) / (t[i+1] - t[i])
    pass

def check_acceleration_limits(q_traj, t_traj, qdd_max):
    """
    Check if trajectory respects acceleration limits.
    
    Returns:
    --------
    is_valid : bool
    qdd_traj : ndarray
        Computed accelerations
    violations : list
    """
    # TODO: Implement acceleration checking
    pass

# Test with a simple trajectory
t_test = np.linspace(0, 2, 100)
q_test = np.column_stack([
    np.sin(2*np.pi*t_test),
    0.5*np.cos(np.pi*t_test),
    0.3*np.sin(3*np.pi*t_test),
    np.zeros_like(t_test)
])

# TODO: Check limits and plot results

### Exercise 5.2: Time-Optimal Trajectory Scaling

Given a desired path, find the minimum time to execute it while respecting all constraints.

In [ ]:
def time_optimal_scaling(waypoints, qd_max, qdd_max):
    """
    Find minimum execution time for trajectory through waypoints.
    
    Parameters:
    -----------
    waypoints : ndarray (M, n_joints)
        Joint space waypoints
    qd_max, qdd_max : ndarray (n_joints,)
        Velocity and acceleration limits
    
    Returns:
    --------
    t_optimal : float
        Minimum time
    q_traj : ndarray
        Optimal trajectory
    t_traj : ndarray
        Time vector
    """
    # TODO: Implement time-optimal scaling
    # Strategy:
    # 1. For each segment, compute minimum time based on distance and limits
    # 2. Use trapezoidal or S-curve velocity profile
    # 3. Ensure smooth blending at waypoints
    pass

# Test case: Pick-and-place waypoints
waypoints_pick = np.array([
    [0, np.deg2rad(-45), np.deg2rad(45), 0],      # Home
    [np.deg2rad(30), np.deg2rad(-20), np.deg2rad(30), 0],  # Approach
    [np.deg2rad(30), np.deg2rad(-10), np.deg2rad(20), 0],  # Grasp
    [np.deg2rad(30), np.deg2rad(-30), np.deg2rad(50), 0],  # Lift
])

# TODO: Find optimal time and generate trajectory

## Part 2: Singularity Avoidance (2 hours)

### Theory

**Singularities** occur when the robot Jacobian loses rank, causing:
- Loss of mobility in certain directions
- Infinite joint velocities required for finite Cartesian velocity
- Poor controllability

**Detection**: Compute manipulability index
$$w = \sqrt{\det(J J^T)}$$

**Avoidance strategies**:
1. Trajectory modification to stay away from singularities
2. Damped least squares (DLS) for IK near singularities
3. Task prioritization

In [ ]:
def compute_manipulability(robot, q):
    """
    Compute manipulability index at configuration q.
    
    Parameters:
    -----------
    robot : DHRobot
    q : ndarray (n_joints,)
        Joint configuration
    
    Returns:
    --------
    w : float
        Manipulability index (0 = singular, higher = better)
    """
    # TODO: Implement manipulability computation
    # Hint: Use robot.jacob0(q) to get Jacobian
    pass

def is_near_singularity(robot, q, threshold=0.01):
    """
    Check if configuration is near a singularity.
    """
    w = compute_manipulability(robot, q)
    return w < threshold

# Test: Find singularities for OpenManipulator-X
# Common singular configurations:
q_singular_1 = np.array([0, 0, 0, 0])  # Fully extended
q_singular_2 = np.array([0, np.deg2rad(90), np.deg2rad(-90), 0])  # Folded

# TODO: Compute and compare manipulability

### Exercise 5.3: Manipulability Map

Create a 2D map showing manipulability in the workspace.

In [ ]:
def create_manipulability_map(robot, x_range, y_range, z_height, resolution=20):
    """
    Create manipulability map at given height.
    
    Parameters:
    -----------
    robot : DHRobot
    x_range, y_range : tuple
        (min, max) for X and Y coordinates
    z_height : float
        Fixed Z coordinate
    resolution : int
        Grid resolution
    
    Returns:
    --------
    X, Y : ndarray
        Meshgrid coordinates
    W : ndarray
        Manipulability values (NaN where unreachable)
    """
    # TODO: Implement manipulability mapping
    # For each (x, y) point:
    # 1. Create target pose at (x, y, z_height)
    # 2. Solve IK
    # 3. Compute manipulability
    # 4. Store in grid
    pass

# TODO: Create and plot manipulability map

### Exercise 5.4: Singularity-Robust IK

Implement damped least squares (DLS) inverse kinematics.

In [ ]:
def ik_damped_least_squares(robot, T_target, q0, lambda_damp=0.01, max_iter=100, tol=1e-4):
    """
    Damped Least Squares IK (numerical method).
    
    Δq = J^T (JJ^T + λ²I)^{-1} Δx
    
    Parameters:
    -----------
    robot : DHRobot
    T_target : SE3
        Target pose
    q0 : ndarray
        Initial guess
    lambda_damp : float
        Damping factor (larger = more robust to singularities, less accurate)
    
    Returns:
    --------
    q_sol : ndarray
        Solution (or best attempt)
    success : bool
    iterations : int
    """
    # TODO: Implement DLS-IK
    pass

# Compare standard IK vs DLS-IK near singularity
# TODO: Test both methods on challenging poses

## Part 3: Collision Detection and Avoidance (2 hours)

### Theory

For the manufacturing cell, we need to avoid collisions between:
- Robot links and itself (self-collision)
- Robot and workspace obstacles (conveyor, bins, inspection station)
- Robot and parts being carried

**Simplified approach**: Capsule-based collision detection
- Model each robot link as a capsule (line segment + radius)
- Model obstacles as boxes or cylinders
- Check minimum distance between primitives

In [ ]:
class Obstacle:
    """Base class for workspace obstacles."""
    pass

class BoxObstacle(Obstacle):
    def __init__(self, center, size):
        """
        Parameters:
        -----------
        center : array (3,)
            Box center position [x, y, z]
        size : array (3,)
            Box dimensions [width, depth, height]
        """
        self.center = np.array(center)
        self.size = np.array(size)
    
    def contains_point(self, point):
        """Check if point is inside box."""
        half_size = self.size / 2
        return np.all(np.abs(point - self.center) <= half_size)
    
    def distance_to_point(self, point):
        """Compute minimum distance from point to box surface."""
        # TODO: Implement distance calculation
        pass

class CylinderObstacle(Obstacle):
    def __init__(self, base_center, height, radius):
        self.base_center = np.array(base_center)
        self.height = height
        self.radius = radius
    
    def distance_to_point(self, point):
        """Compute minimum distance from point to cylinder surface."""
        # TODO: Implement distance calculation
        pass

# Define manufacturing cell obstacles
obstacles = [
    BoxObstacle(center=[0.0, 0.0, -0.05], size=[1.0, 0.2, 0.1]),  # Conveyor belt
    BoxObstacle(center=[0.25, 0.15, 0.05], size=[0.15, 0.15, 0.1]), # Bin A
    BoxObstacle(center=[0.25, -0.15, 0.05], size=[0.15, 0.15, 0.1]), # Bin B
    CylinderObstacle(base_center=[0.15, 0.0, 0.0], height=0.15, radius=0.03), # Inspection station
]

### Exercise 5.5: Collision Checking

Implement collision detection for robot configurations.

In [ ]:
def get_link_positions(robot, q):
    """
    Get positions of all link endpoints.
    
    Returns:
    --------
    positions : list of ndarray
        Position of each link end [x, y, z]
    """
    positions = [np.array([0, 0, 0])]  # Base
    
    for i in range(len(q)):
        T = robot.A(range(i+1), q)
        positions.append(T.t)
    
    return positions

def check_collision(robot, q, obstacles, safety_margin=0.02):
    """
    Check if configuration q causes collision.
    
    Parameters:
    -----------
    robot : DHRobot
    q : ndarray
        Joint configuration
    obstacles : list of Obstacle
    safety_margin : float
        Additional clearance required (meters)
    
    Returns:
    --------
    collision : bool
        True if collision detected
    min_distance : float
        Minimum distance to any obstacle
    """
    # TODO: Implement collision checking
    # 1. Get all link positions
    # 2. For each link endpoint, check distance to all obstacles
    # 3. Return collision if any distance < safety_margin
    pass

def is_trajectory_collision_free(robot, q_traj, obstacles, safety_margin=0.02):
    """
    Check if entire trajectory is collision-free.
    """
    for q in q_traj:
        collision, _ = check_collision(robot, q, obstacles, safety_margin)
        if collision:
            return False
    return True

# Test collision detection
# TODO: Create test configurations (some colliding, some not)

### Exercise 5.6: Collision Avoidance in Path Planning

Modify waypoint generation to avoid obstacles.

In [ ]:
def plan_collision_free_path(robot, q_start, q_goal, obstacles, n_samples=50):
    """
    Plan path from start to goal avoiding obstacles.
    
    Uses simple approach: try via-points at different heights.
    
    Parameters:
    -----------
    robot : DHRobot
    q_start, q_goal : ndarray
        Start and goal configurations
    obstacles : list
    n_samples : int
        Number of points to check along path
    
    Returns:
    --------
    waypoints : list of ndarray
        Joint space waypoints (collision-free)
    success : bool
    """
    # TODO: Implement collision-free path planning
    # Simple strategy:
    # 1. Try direct linear interpolation in joint space
    # 2. If collision, try with intermediate waypoint at higher Z
    # 3. Check each segment for collisions
    pass

# Test case: Plan pick-to-place trajectory avoiding conveyor
# TODO: Implement and visualize

## Part 4: Energy-Efficient Trajectories (2 hours)

### Theory

Energy consumption is important for:
- Operating costs (electricity)
- Heat generation (affects precision)
- Sustainability (Industry 4.0 goal)

**Energy model** (simplified):
$$E = \int_0^T \sum_{i=1}^n \tau_i(t) \dot{q}_i(t) \, dt$$

Where torque: $\tau = M(q)\ddot{q} + C(q,\dot{q})\dot{q} + g(q)$

**Optimization goal**: Minimize energy while meeting task requirements.

In [ ]:
def estimate_trajectory_energy(robot, q_traj, qd_traj, qdd_traj, dt):
    """
    Estimate energy consumption of trajectory.
    
    Simplified model (no full dynamics):
    E ≈ Σ |qdd|² + α*Σ|qd|² (kinetic + resistive losses)
    
    Parameters:
    -----------
    robot : DHRobot
    q_traj : ndarray (N, n_joints)
    qd_traj : ndarray (N, n_joints)
        Joint velocities
    qdd_traj : ndarray (N, n_joints)
        Joint accelerations
    dt : float
        Time step
    
    Returns:
    --------
    energy : float
        Total energy (arbitrary units)
    energy_per_joint : ndarray
        Energy breakdown by joint
    """
    alpha = 0.1  # Resistance coefficient
    
    # Acceleration energy (proportional to torque squared)
    E_acc = np.sum(qdd_traj**2) * dt
    
    # Velocity energy (friction/resistance)
    E_vel = alpha * np.sum(qd_traj**2) * dt
    
    total_energy = E_acc + E_vel
    
    # Per-joint breakdown
    energy_per_joint = np.sum(qdd_traj**2, axis=0) * dt + alpha * np.sum(qd_traj**2, axis=0) * dt
    
    return total_energy, energy_per_joint

# TODO: Test with different trajectory types

### Exercise 5.7: Trajectory Comparison

Compare energy consumption of different interpolation methods.

In [ ]:
def compare_trajectory_methods(robot, waypoints, duration):
    """
    Compare different trajectory generation methods.
    
    Methods to compare:
    1. Linear interpolation
    2. Cubic spline
    3. Quintic spline
    4. Trapezoidal velocity profile
    
    Returns:
    --------
    results : dict
        {method_name: {energy, max_velocity, max_acceleration, ...}}
    """
    # TODO: Implement comparison
    pass

# Test waypoints (pick-and-place)
waypoints_test = np.array([
    [0, np.deg2rad(-45), np.deg2rad(45), 0],
    [np.deg2rad(30), np.deg2rad(-20), np.deg2rad(30), 0],
    [np.deg2rad(-30), np.deg2rad(-20), np.deg2rad(30), 0],
    [0, np.deg2rad(-45), np.deg2rad(45), 0],
])

# TODO: Run comparison and plot results

### Exercise 5.8: Energy-Optimal Speed Scaling

Find the execution speed that minimizes energy for a given path.

In [ ]:
def find_energy_optimal_duration(robot, waypoints, duration_range=(1.0, 10.0), n_samples=20):
    """
    Find duration that minimizes energy consumption.
    
    Trade-off:
    - Faster → Higher accelerations → More energy
    - Slower → Longer time against gravity → More energy
    
    Parameters:
    -----------
    robot : DHRobot
    waypoints : ndarray
    duration_range : tuple
        (min_duration, max_duration) to test
    n_samples : int
        Number of durations to try
    
    Returns:
    --------
    optimal_duration : float
    energy_vs_duration : dict
        Duration vs energy data for plotting
    """
    # TODO: Implement energy optimization
    pass

# TODO: Find optimal duration and plot energy curve

## Integration Exercise: Optimized Manufacturing Cell Operation

### Exercise 5.9: Complete Optimized Cycle

Combine all advanced control features into a complete pick-inspect-place cycle.

In [ ]:
class OptimizedManufacturingCell:
    """
    Advanced manufacturing cell with:
    - Velocity/acceleration limiting
    - Singularity avoidance
    - Collision avoidance
    - Energy optimization
    """
    
    def __init__(self, robot, obstacles):
        self.robot = robot
        self.obstacles = obstacles
        
        # Performance metrics
        self.total_energy = 0
        self.total_time = 0
        self.collision_checks = 0
        self.singularity_avoidances = 0
    
    def plan_optimized_pick_place(self, part_pose, bin_pose):
        """
        Plan complete pick-place operation with all optimizations.
        
        Steps:
        1. Generate waypoints (home → pick → inspect → place → home)
        2. Check for singularities, modify if needed
        3. Check for collisions, add via-points if needed
        4. Optimize timing for energy efficiency
        5. Generate smooth trajectory respecting all limits
        
        Returns:
        --------
        trajectory : dict
            {t, q, qd, qdd, energy, warnings}
        """
        # TODO: Implement complete optimized planning
        pass
    
    def execute_and_monitor(self, trajectory):
        """
        Simulate trajectory execution with monitoring.
        """
        # TODO: Implement execution simulation
        pass
    
    def get_performance_report(self):
        """
        Generate performance statistics.
        """
        return {
            'total_energy': self.total_energy,
            'total_time': self.total_time,
            'average_cycle_time': self.total_time / max(1, self.cycles_completed),
            'energy_per_cycle': self.total_energy / max(1, self.cycles_completed),
            'collision_checks': self.collision_checks,
            'singularity_avoidances': self.singularity_avoidances,
        }

# TODO: Create optimized cell and run test cycles

## Summary and Deliverables

### What You Should Have Learned:

1. **Velocity/Acceleration Control**: How to verify and enforce kinematic limits
2. **Time Optimization**: Finding minimum execution time for trajectories
3. **Singularity Analysis**: Detecting and avoiding singular configurations
4. **Collision Detection**: Checking robot-environment interactions
5. **Collision Avoidance**: Planning paths that avoid obstacles
6. **Energy Optimization**: Minimizing power consumption

### Integration with SAE Project:

Apply these techniques to your manufacturing cell:
- Ensure all trajectories respect OpenManipulator-X limits
- Add collision checking with conveyor, bins, inspection station
- Optimize cycle time while maintaining safety
- Track and minimize energy consumption
- Add manipulability monitoring to HMI

### For the Report:

Include a new section on **Advanced Control**:
- Explain your collision avoidance strategy
- Show manipulability analysis of your workspace
- Present energy optimization results
- Compare different trajectory methods
- Discuss trade-offs (speed vs energy vs safety)

---

**Next Session**: Performance analysis and system validation